In [190]:
"""Importowanie potrzebnych w projekcie bibliotek"""
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



"""
Poniższa metoda to implementacja środków ciężkości z wykorzystaniem funkcji cv2.moments()
W dalszej części kodu znajduje się implementacja z wykorzystaniem własnej metody

def Moment_I(self):
        Operator = self.binImage.copy()
        M = cv2.moments(Operator)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return [cX, cY]
"""

'\nPoniższa metoda to implementacja środków ciężkości z wykorzystaniem funkcji cv2.moments()\nW dalszej części kodu znajduje się implementacja z wykorzystaniem własnej metody\n\ndef Moment_I(self):\n        Operator = self.binImage.copy()\n        M = cv2.moments(Operator)\n        cX = int(M["m10"] / M["m00"])\n        cY = int(M["m01"] / M["m00"])\n        return [cX, cY]\n'

In [191]:
"""
Deklaracja klasy ImageShape zawierającej metody służące do badania współczynników kształtu figury na obrazie
"""
class ImageShape:
    def __init__(self, srcImage):
        self.baseImage = srcImage.copy()
        self.grayImage = cv2.cvtColor(self.baseImage, cv2.COLOR_BGR2GRAY)
        (self.thr, self.binImage) = cv2.threshold(self.grayImage, 0, 1, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    """
    Implementacja metody szukającej punktu startowego dla FCC
    """
    def getStartPos(self, srcImage):
        for i in range(srcImage.shape[0]):
            for j in range(srcImage.shape[1]):
                getVal = srcImage[i,j]
                if(0 < getVal):
                    startPosition = (i, j)
                    return startPosition

    """
    Implementacja FCC z poprzedniego laboratorium w celu obliczania obwodu figury
    """
    def Perimeter(self, srcImage):
        directions = [  5,  6,  7,
                        4,      0,
                        3,  2,  1]
        
        change_j =   [  -1,  0,  1,
                        -1,      1,
                        -1,  0,  1]
        
        change_i =   [  -1, -1, -1,
                         0,      0,
                         1,  1,  1]
        
        dir2idx = dict(zip(directions, range(len(directions))))
        start_point = self.getStartPos(srcImage)
        curr_point = start_point
        border = []
        chain = []
        for direction in directions:
            idx = dir2idx[direction]
            new_point = (start_point[0]+change_i[idx], start_point[1]+change_j[idx])
            if srcImage[new_point] != 0: # if is ROI
                border.append(new_point)
                chain.append(direction)
                curr_point = new_point
                break
        
        count = 0
        while curr_point != start_point:
            b_direction = (direction + 5) % 8 
            dirs_1 = range(b_direction, 8)
            dirs_2 = range(0, b_direction)
            dirs = []
            dirs.extend(dirs_1)
            dirs.extend(dirs_2)
            for direction in dirs:
                idx = dir2idx[direction]
                new_point = (curr_point[0]+change_i[idx], curr_point[1]+change_j[idx])
                if srcImage[new_point] != 0: # if is ROI
                    border.append(new_point)
                    chain.append(direction)
                    curr_point = new_point
                    break
            if count == 1000: break
            count += 1
        
        straight_lines = [0, 2, 4, 6]
        straight_count = 0
        curve_count = 0
        for l in chain:
            if l in straight_lines:
                straight_count += 1
            else:
                curve_count += 1
        
        return (straight_count + (curve_count*np.sqrt(2)))

    """
    Metoda wyliczająca wartości momentów I-go rzędu (środek ciężkości)
    """
    def Moment_I(self):
        SumPosX = 0
        SumPosY = 0
        for i in range(self.binImage.shape[0]):
            for j in range(self.binImage.shape[1]):
                if(self.binImage[i,j] == 1):
                    SumPosX += j
                    SumPosY += i
        cX = (1/np.sum(self.binImage))*SumPosX
        cY = (1/np.sum(self.binImage))*SumPosY
        return [cX, cY]
    
    """
    Metoda wyliczająca wartości momentów II-go rzędu
    """
    def Moment_II(self):
        [M1X, M1Y] = self.Moment_I()
        SumX = 0
        SumY = 0
        SumXY = 0
        for i in range(self.binImage.shape[0]):
            for j in range(self.binImage.shape[1]):
                if(self.binImage[i,j] == 1):
                    SumX += np.power((j - M1X),2)
                    SumY += np.power((i - M1Y),2)
                    SumXY += ((j - M1X)*(i - M1Y))

        M2X = (1/np.sum(self.binImage))*SumX
        M2Y = (1/np.sum(self.binImage))*SumY
        M2XY = (1/np.sum(self.binImage))*SumXY
        return [M2X, M2Y, M2XY]
    
    """Metoda wyliczająca współczynnik Fereta"""
    def Feret_coefficient(self):
        Lh = 0
        Lf = 0
        Rf = 0
        return Rf
    
    """Metoda wyliczająca współczynniki cyrkularności"""
    def Circle_coefficient(self):
        S = np.sum(self.binImage)
        L = self.Perimeter(self.binImage)
        RC1 = 2 * np.sqrt((S/np.pi))
        RC2 = L/np.pi
        return [RC1, RC2]
    
    def Malinowska_coefficient(self):
        L = self.Perimeter(self.binImage)
        S = np.sum(self.binImage)
        RM = (L/(2*np.sqrt(np.pi*S)))-1
        return RM
    
    def Dimensionless_Shape_Factor(self):
        L = self.Perimeter(self.binImage)
        S = np.sum(self.binImage)
        RS = np.power(L,2)/(4*np.pi*S)
        return RS
        


In [192]:
Image = ImageShape(cv2.imread("./circle.bmp"))
print(Image.Dimensionless_Shape_Factor())

1.1062514877643308
